In [1]:
import numpy as np
import pandas as pd
import warnings,gc,os,time,math
warnings.filterwarnings('ignore')
from functools import reduce
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [2]:
def check_df(df):
    df=df.drop_duplicates()
    print(df.shape)
    print(df[['VIN','修理日期']].drop_duplicates().shape)
    print(df['VIN'].nunique())
    
def check_run_time(s):
    r=(time.time()-s)/3600
    return "Run Hour: %f"%r

def get_user(df):
    df['VIN']=df['VIN'].apply(str)
    df['车身型式']=df['VIN'].apply(lambda x:x[3])
    df['变速器']=df['VIN'].apply(lambda x:x[4])
    df['乘员保护系统']=df['VIN'].apply(lambda x:x[5])
    df['车辆等级']=df['VIN'].apply(lambda x:x[6:8])
    df['生产年份']=df['VIN'].apply(lambda x:x[9])
    df['装配厂']=df['VIN'].apply(lambda x:x[10])
    for col in ['车身型式', '变速器', '乘员保护系统', '车辆等级', '生产年份', '装配厂']:
        df[col]=df[col].astype(str)
    df['用户属性']=list(map(lambda a,b,c,d,e,f:a+','+b+','+c+','+d+','+e+','+f,df['车身型式'],df['变速器'],
                        df['乘员保护系统'],df['车辆等级'],df['生产年份'],df['装配厂']))
    return df


def get_last_run(df):
    df['修理日期']=pd.to_datetime(df['修理日期'])
    df['购车时间']=pd.to_datetime(df['购车时间'])
    df=df.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    df['修理日期_last']=df.groupby('VIN')['修理日期'].shift(-1)
    df['公里数_last']=df.groupby('VIN')['公里数'].shift(-1)
    df.loc[df['修理日期_last'].isnull(),'修理日期_last']=df['购车时间']
    df.loc[df['公里数_last'].isnull(),'公里数_last']=0
    df['上下天数差']=(df['修理日期']-df['修理日期_last']).apply(lambda x:x.days)
    df['上下公里数差']=(df['公里数']-df['公里数_last'])
    return df

### <font color=red>第一部分:数据合并,加,迭代

In [3]:
filename='ud20201019'
num='_34' 
today='20201023'

datap='/home/jupyter/LQ/VW/data/'
new_datap='/home/jupyter/LCG/dazhong/V2/%s/'%(filename) #读取阿桂的数据
savep='/home/jupyter/LQ/VW/20191217_VW_下次保养里程预测/20201023自动化更新数据/%s%s/'%(today,str(num))

if os.path.exists(savep):
    print('True')
else:
    os.mkdir(savep)
    
store_num=num.replace('_','')
store_num=int(store_num)
print(store_num)

if (store_num<=20):
    baoyang_id_max=10
elif (store_num>20)&(store_num<=40):
    baoyang_id_max=11
elif (store_num>40)&(store_num<=50):
    baoyang_id_max=12
elif store_num>50:
    baoyang_id_max=13
    
print('biggest batch :',baoyang_id_max)

s1=time.time()

True
30
biggest batch : 11


In [4]:
updated_savep='/home/jupyter/LQ/VW/updated_data_toChongGui/0_月更/'

### <font color=red>得到二个结果

In [5]:
df=pd.read_pickle(savep+'final_zhudan.pkl')
print(df.shape)
print(df[['VIN','修理日期']].drop_duplicates().shape)
print(df['VIN'].nunique())
print(list(df.columns))

(3174254, 30)
(3174254, 2)
804143
['VIN', 'brand', '一般维修', '事故车维修', '二手车认证维修', '保养状态', '修理日期', '修理类型', '修理类型_new', '公里数', '内部车辆维修', '在店状态', '工时费', '常去经销商', '常规保养', '普通索赔', '服务活动', '经销商代码', '维修金额', '维修金额_折前', '购车时间', '附件安装', '零件费', '首保', '人群_一级分类', '车龄', '车龄bin', '常规次数', '首保次数', '人群_二级分类']


In [6]:
baoyang=df[(df['首保']==1)|(df['常规保养']==1)]
print(baoyang.shape)

(3055155, 30)


In [7]:
baoyang=get_last_run(baoyang)
print(baoyang['上下天数差'].min())
print(baoyang['上下公里数差'].min())

8
-494274.0


In [8]:
baoyang1=baoyang[baoyang['上下公里数差']>0]
baoyang_info=baoyang1.groupby('VIN')['上下天数差','上下公里数差'].agg({'max','min','median','mean'})
baoyang_info.columns=['_'.join(x) for x in baoyang_info.columns.tolist()]
baoyang_info=baoyang_info.reset_index()
print(baoyang_info.head())

                 VIN  上下天数差_min  上下天数差_median  上下天数差_max  上下天数差_mean  \
0  LSV00000000000000         20          32.0       3418  277.857143   
1  LSV0F6180F2205902        140         280.0        420  280.000000   
2  LSV0F6180F2261984         60          74.0         88   74.000000   
3  LSV0F6180F2279269         85         195.0        532  270.666667   
4  LSV0F6180G2034375         57         110.0        239  119.000000   

   上下公里数差_min  上下公里数差_median  上下公里数差_max   上下公里数差_mean  
0        10.0        10474.5     53987.0  19539.214286  
1      5128.0        11181.5     17235.0  11181.500000  
2      5286.0         5667.0      6048.0   5667.000000  
3      5418.0         9922.0     22724.0  12688.000000  
4      5007.0         7621.0     14963.0   8064.888889  


In [9]:
modeldf_1=pd.DataFrame()
for i in range(2,baoyang_id_max+1):
    print(i)
    tmp=pd.read_pickle(savep+'%d_model_result.pkl'%i)
    modeldf_1=pd.concat([modeldf_1,tmp],axis=0)
print(modeldf_1.shape)

2
3
4
5
6
7
8
9
10
11
(549622, 51)


In [10]:
cols=['VIN', '购车时间', '在店状态', '保养状态', '人群_一级分类', '人群_二级分类', 
      '期间最后一次保养天数', '期间最后一次保养公里数','天数差_next_fin','公里数差_next_fin',
      '日均_next_fin', '天数差',
      '下一次保养公里数', '公里数差', '最终日均', '下一次保养日期', '最后一次保养日期',
      '最后一次保养公里数']
modeldf=modeldf_1[cols]
print(modeldf.shape)
print(modeldf['VIN'].nunique())

(549622, 18)
549622


In [11]:
modeldf=modeldf.merge(baoyang_info,on=['VIN'],how='left')
print(modeldf.shape)
print(modeldf['VIN'].nunique())

(549622, 26)
549622


In [12]:
print('模型中极端值放到规则中')
print('1.自身平滑掉,天数差_next_fin<期间最后一次保养天数')
modeldf.loc[modeldf['天数差_next_fin']<=modeldf['期间最后一次保养天数'],'put_in_rule']=1

模型中极端值放到规则中
1.自身平滑掉,天数差_next_fin<期间最后一次保养天数


In [13]:
print('2.自身平滑掉,公里数差_next_fin<期间最后一次保养公里数')
modeldf['公里数差_next_fin']=modeldf['公里数差_next_fin'].astype(float)
modeldf['期间最后一次保养公里数']=modeldf['期间最后一次保养公里数'].astype(float)
modeldf.loc[modeldf['公里数差_next_fin']<=modeldf['期间最后一次保养公里数'],'put_in_rule']=1

2.自身平滑掉,公里数差_next_fin<期间最后一次保养公里数


In [14]:
print('3.自身平滑掉,天数差<天数差_min*0.5')
modeldf.loc[(modeldf['天数差']<modeldf['上下天数差_min']*0.5)&(modeldf['天数差']<=15),'put_in_rule']=1

3.自身平滑掉,天数差<天数差_min*0.5


In [15]:
print('4.自身平滑掉,天数差>天数差_max*1.5')
modeldf.loc[(modeldf['天数差']>modeldf['上下天数差_max']*2)&(modeldf['天数差']>=365*3),'put_in_rule']=1

4.自身平滑掉,天数差>天数差_max*1.5


In [16]:
# print('5.自身平滑掉,公里数差<公里数差_min*0.5')
# modeldf.loc[modeldf['公里数差']<modeldf['上下公里数差_min']*0.5,'put_in_rule_5']=1
# print(len(modeldf[modeldf['put_in_rule_5']==1]))

# print('6.自身平滑掉,公里数差>公里数差_max*1.5')
# modeldf.loc[modeldf['公里数差']>modeldf['上下公里数差_max']*2,'put_in_rule_6']=1
# print(len(modeldf[modeldf['put_in_rule_6']==1]))

In [17]:
put_rule=modeldf[modeldf['put_in_rule']==1]
print(put_rule.shape)
print(put_rule['VIN'].nunique())

print(put_rule['VIN'].nunique()/modeldf['VIN'].nunique())

model=modeldf[~(modeldf['VIN'].isin(put_rule['VIN']))]
model['mark']='模型预测'
print(model['VIN'].nunique())

(3377, 27)
3377
0.00614422275673099
546245


In [19]:
ruledf=pd.DataFrame()
for i in range(8):
    tmp=pd.read_pickle(savep+'%d_rule_test.pkl'%i)
    ruledf=pd.concat([ruledf,tmp],axis=0)
print(ruledf.shape)

(850310, 9)


In [20]:
ruledf.columns=['VIN', '最后一次保养日期', '最后一次保养公里数', '天数差', '公里数差',
               '最终日均', '下一次保养日期', '下一次保养公里数','mark']
print(ruledf.shape)
print(ruledf['VIN'].nunique())
print(pd.isnull(ruledf).sum())

(850310, 9)
850310
VIN          0
最后一次保养日期     0
最后一次保养公里数    0
天数差          0
公里数差         0
最终日均         0
下一次保养日期      0
下一次保养公里数     0
mark         0
dtype: int64


In [21]:
rule=ruledf[~(ruledf['VIN'].isin(set(model['VIN'])))]
print(rule.shape)
print(rule['VIN'].nunique())

(304065, 9)
304065


In [22]:
rule=rule.merge(baoyang_info,on='VIN',how='left')

In [23]:
same_cols=[x for x in model.columns.tolist() if x in rule.columns.tolist()]
print(len(same_cols))
print(same_cols)

17
['VIN', '天数差', '下一次保养公里数', '公里数差', '最终日均', '下一次保养日期', '最后一次保养日期', '最后一次保养公里数', '上下天数差_min', '上下天数差_median', '上下天数差_max', '上下天数差_mean', '上下公里数差_min', '上下公里数差_median', '上下公里数差_max', '上下公里数差_mean', 'mark']


In [24]:
pred=pd.concat([model[same_cols],rule[same_cols]],axis=0)
pred['下一次保养日期']=pd.to_datetime(pred['下一次保养日期'])
print(pred.shape)
print(pred['VIN'].nunique())
print(pred.columns.tolist())

(850310, 17)
850310
['VIN', '天数差', '下一次保养公里数', '公里数差', '最终日均', '下一次保养日期', '最后一次保养日期', '最后一次保养公里数', '上下天数差_min', '上下天数差_median', '上下天数差_max', '上下天数差_mean', '上下公里数差_min', '上下公里数差_median', '上下公里数差_max', '上下公里数差_mean', 'mark']


In [25]:
def get_iterated(pred):

    pred1=pred[pred['下一次保养日期']>today]
    pred1['下一次保养日期_今天之后']=pred1['下一次保养日期']
    pred1['下一次保养公里数_今天之后']=pred1['下一次保养公里数']
    pred1['迭代次数']=0
    print(pred1.shape)
    print(pred1['VIN'].nunique())

    pred2=pred[pred['下一次保养日期']<=today]
    pred2['今天与预测第一次进店的天数差']=(pd.to_datetime(today)-pred2['下一次保养日期']).apply(lambda x:x.days)
    pred2['迭代次数']=pred2['今天与预测第一次进店的天数差']/pred2['天数差']
    pred2['迭代次数']=pred2['迭代次数'].apply(lambda x:math.ceil(x))
    pred2['下一次保养日期_今天之后']=list(map(lambda x,y,z:x+relativedelta(days=math.ceil(y*z)),pred2['下一次保养日期'],pred2['天数差'],pred2['迭代次数']))
    pred2['下一次保养公里数_今天之后']=list(map(lambda x,y,z:x+y*z,pred2['下一次保养公里数'],pred2['公里数差'],pred2['迭代次数']))
    pred2['下一次保养公里数_今天之后']=pred2['下一次保养公里数_今天之后'].apply(lambda x:math.ceil(x))
    print(pred2.shape)
    print(pred2['VIN'].nunique())

    same_cols=[x for x in pred1.columns.tolist() if x in pred2.columns.tolist()]
    print(same_cols)

    new_pred=pd.concat([pred1[same_cols],pred2[same_cols]],axis=0)

    return new_pred

In [26]:
new_pred=get_iterated(pred)
print(new_pred['VIN'].nunique())

(156326, 20)
156326
(693984, 21)
693984
['VIN', '天数差', '下一次保养公里数', '公里数差', '最终日均', '下一次保养日期', '最后一次保养日期', '最后一次保养公里数', '上下天数差_min', '上下天数差_median', '上下天数差_max', '上下天数差_mean', '上下公里数差_min', '上下公里数差_median', '上下公里数差_max', '上下公里数差_mean', 'mark', '下一次保养日期_今天之后', '下一次保养公里数_今天之后', '迭代次数']
850310


In [27]:
def get_another_date(new_pred):
    org_zhudan=pd.read_pickle(savep+'zhudan_merged_org.pkl')
    print(org_zhudan.columns.tolist())

    org_baoyang=org_zhudan[(org_zhudan['首保']==1)|(org_zhudan['常规保养']==1)]
    org_baoyang['修理日期']=pd.to_datetime(org_baoyang['修理日期'])
    org_baoyang=org_baoyang.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)

    last_baoyang=org_baoyang.drop_duplicates(['VIN'],keep='first').reset_index(drop=True)
    last_baoyang=last_baoyang[['VIN','修理日期','公里数']].drop_duplicates().reset_index(drop=True)
    last_baoyang.columns=['VIN','实际最后一次保养日期','实际最后一次保养公里数']

    new_pred=new_pred.merge(last_baoyang,on=['VIN'],how='left')

    print('提取购车时间')
    veh=pd.read_pickle(savep+'vehicle_info.pkl')
    veh.loc[veh['新购车时间'].isnull(),'新购车时间']=veh[veh['新购车时间'].isnull()]['购车时间']
    veh1=veh[['VIN','新购车时间']].drop_duplicates().reset_index(drop=True).rename(columns={'新购车时间':'购车时间'})
    new_pred=new_pred.merge(veh1,on=['VIN'],how='left')
    new_pred['购车时间']=pd.to_datetime(new_pred['购车时间'])
    new_pred.loc[new_pred['实际最后一次保养日期'].isnull(),'实际最后一次保养日期']=new_pred[new_pred['实际最后一次保养日期'].isnull()]['购车时间']
    new_pred.loc[new_pred['实际最后一次保养公里数'].isnull(),'实际最后一次保养公里数']=0
    print(new_pred['VIN'].nunique())
    print(pd.isnull(new_pred[['VIN','实际最后一次保养日期','实际最后一次保养公里数']]).sum())

    new_pred['下次1万公里保养公里数']=new_pred['实际最后一次保养公里数']+10000

    new_pred.loc[new_pred['最终日均']<=new_pred['最终日均'].quantile(0.01),'最终日均']=new_pred['最终日均'].quantile(0.01)
    new_pred.loc[new_pred['最终日均']>=new_pred['最终日均'].quantile(0.99),'最终日均']=new_pred['最终日均'].quantile(0.99)

    new_pred['下次1万公里天数间隔']=10000/new_pred['最终日均']
    new_pred['下次1万公里天数间隔']=new_pred['下次1万公里天数间隔'].apply(lambda x:math.ceil(x))
    print(pd.isnull(new_pred[['VIN','下次1万公里天数间隔']]).sum())

    new_pred['实际最后一次保养日期']=pd.to_datetime(new_pred['实际最后一次保养日期'])
    new_pred['下次1万公里保养日期']=list(map(lambda x,y:x+relativedelta(days=y),new_pred['实际最后一次保养日期'],new_pred['下次1万公里天数间隔']))
    new_pred=new_pred.drop_duplicates().reset_index(drop=True)
    print(new_pred.shape)
    print(new_pred['VIN'].nunique())
    print(new_pred.columns.tolist())

    new_pred=new_pred.rename(columns={'下一次保养日期':'下次保养时间_按客户习惯',
                          '下一次保养公里数':'下次保养公里数_按客户习惯',
                      '下一次保养日期_今天之后': '下次保养时间_当前时间之后_按客户习惯',
                      '下一次保养公里数_今天之后': '下次保养里程_当前时间之后_按客户习惯',
                      '下次1万公里保养日期': '下次保养时间_当前时间之后_按官方建议周期',
                      '下次1万公里保养公里数': '下次保养里程_当前时间之后_按官方建议周期'})

    return new_pred

In [28]:
new_pred_1 = get_another_date(new_pred)

['VIN', '修理日期', '公里数', '修理类型', '经销商代码', '结算单号', '委托书号', '维修金额', '维修金额_折前', '工时费', '零件费', '常规保养', '首保', '附件安装', '事故车维修', '一般维修', '普通索赔', '服务活动', '二手车认证维修', '内部车辆维修']
提取购车时间
850310
VIN            0
实际最后一次保养日期     0
实际最后一次保养公里数    0
dtype: int64
VIN           0
下次1万公里天数间隔    0
dtype: int64
(850310, 26)
850310
['VIN', '天数差', '下一次保养公里数', '公里数差', '最终日均', '下一次保养日期', '最后一次保养日期', '最后一次保养公里数', '上下天数差_min', '上下天数差_median', '上下天数差_max', '上下天数差_mean', '上下公里数差_min', '上下公里数差_median', '上下公里数差_max', '上下公里数差_mean', 'mark', '下一次保养日期_今天之后', '下一次保养公里数_今天之后', '迭代次数', '实际最后一次保养日期', '实际最后一次保养公里数', '购车时间', '下次1万公里保养公里数', '下次1万公里天数间隔', '下次1万公里保养日期']


In [29]:
ziduan_name='用于易损件推荐'
new_pred_1.to_pickle(savep+'2个预测结果_%s.pkl'%(ziduan_name))

In [30]:
new_pred_1.to_pickle(updated_savep+'2个预测结果_%s.pkl'%(ziduan_name))

### <font color=red>scp -r /home/jupyter/LQ/VW/updated_data_toChongGui/0_月更 root@192.168.1.253:/home/jupyter/LQ

### <font color=red>第二部分:易损件推荐

#### 陆琴，你那边修改下，易损件推荐删除转向助力液，正时皮带正更新为正时皮带链条

#### 对了，易损件提醒里面，转向助力液删掉，空滤删掉，正式皮带变成正时皮带/链条

In [32]:
def get_car_type(veh):#14
    veh.loc[(veh['一级车系']=='Polo')&(veh['二级车系']=='非电动车'),'推荐车型']='A'
    veh.loc[(veh['一级车系']=='途观')&(veh['二级车系']=='非电动车'),'推荐车型']='B'
    veh.loc[(veh['一级车系']=='途观')&(veh['二级车系']=='电动车'),'推荐车型']='C'
    veh.loc[(veh['一级车系']=='辉昂')&(veh['二级车系']=='非电动车'),'推荐车型']='D'
    veh.loc[(veh['一级车系']=='朗逸')&(veh['二级车系']=='非电动车'),'推荐车型']='E' #有待跟进,是朗逸电动车A版
    veh.loc[(veh['一级车系']=='朗逸')&(veh['二级车系']=='电动车'),'推荐车型']='F'
    veh.loc[(veh['一级车系']=='凌渡')&(veh['二级车系']=='非电动车'),'推荐车型']='G'
    veh.loc[(veh['一级车系']=='帕萨特')&(veh['二级车系']=='非电动车'),'推荐车型']='H'
    veh.loc[(veh['一级车系']=='帕萨特')&(veh['二级车系']=='电动车'),'推荐车型']='I'
    veh.loc[(veh['一级车系']=='途安')&(veh['二级车系']=='非电动车'),'推荐车型']='J'
    veh.loc[(veh['一级车系']=='途昂')&(veh['二级车系']=='非电动车'),'推荐车型']='K'
    veh.loc[(veh['一级车系']=='途铠')&(veh['二级车系']=='非电动车'),'推荐车型']='M'
    veh.loc[(veh['一级车系']=='途岳')&(veh['二级车系']=='非电动车'),'推荐车型']='N'
    veh.loc[(veh['一级车系']=='桑塔纳')&(veh['二级车系']=='非电动车'),'推荐车型']='O'
    return veh

def yanghu_yisun_interval(dt,today1):
    dt['feature_stop_time']=today1
    dt['修理日期']=pd.to_datetime(dt['修理日期'])
    dt['公里数']=dt['公里数'].astype(float)
    
    dt = dt[['VIN','find', '公里数', '修理日期','feature_stop_time']].drop_duplicates()
    dt = dt.sort_values(['VIN','find','修理日期'],ascending = [False,False,False]).reset_index(drop=True)
    dt['last_修理日期'] = dt.groupby(['find','VIN'])['修理日期'].shift(-1)
    dt['last_公里数'] = dt.groupby(['find','VIN'])['公里数'].shift(-1)
    dt['yy_mile_avg'] = dt['公里数'] - dt['last_公里数']
    dt['yy_day_avg'] = (dt['修理日期'] - dt['last_修理日期']).apply(lambda x:x.days)
    dt['yy_last_mile'] = dt['公里数']
    dt['yy_last_time'] = dt['修理日期']
    dt['yy_last_time_interval'] = list(map(lambda x,y:(x-y).days,dt['feature_stop_time'], dt['修理日期']))
    dt = dt.drop_duplicates(['VIN','find'],keep='first').reset_index(drop=True)
    dt = dt[['VIN','find','yy_mile_avg','yy_day_avg','yy_last_mile','yy_last_time','yy_last_time_interval']].drop_duplicates().reset_index(drop=True)
    a = dt.pivot(index = 'VIN',columns = 'find',values = ['yy_mile_avg','yy_day_avg','yy_last_mile','yy_last_time','yy_last_time_interval'])
    a.columns = ['_'.join(x) for x in a.columns] 
    a = a.reset_index()    
    return a

def yisun_recommend(wbb,next_inshop_df,yisun_list,yisun_df,ziduan_name):
    sel_cols_1= []
    for x in ['yy_last_mile_','yy_last_time_','yy_mile_avg_','yy_day_avg_']:
        for y in yisun_list:   
            sel_cols_1.append(x+y)        
    sel_cols_1 = ['VIN']+sel_cols_1
    
    no_in=[x for x in sel_cols_1 if x not in wbb.columns.tolist()]
    for c in no_in:
        wbb[c]=np.nan
    
    yisun_fea_interval = wbb[sel_cols_1]
    
    print('wbb表格的情况 ',wbb.shape)
    wb = wbb.merge(next_inshop_df[['VIN','mile_next','next_repare_time_']],on='VIN',how='left')
    print('wb表格的情况 ',wb.shape)
    wb = wb.merge(yisun_df[['VIN','推荐车型','购车时间','排量','车辆类型','车型档次']],on='VIN',how='left').rename(columns = {'购车时间':'purchase_date'})
    wb['车型档次']=wb['车型档次'].map({'高档':4,'中高档':3,'中低档':2,'低档':1})
    
    wb['next_repare_time_']=pd.to_datetime(wb['next_repare_time_'])
    wb['purchase_date']=pd.to_datetime(wb['purchase_date'])
    wb['mile_next']=wb['mile_next'].apply(float)
    
#     c1='发动机油'
#     wb['yy_day_avg_%s_g'%c1] = 365
#     wb['yy_mile_avg_%s_g'%c1] = np.nan
#     wb.loc[(pd.isna(wb['yy_mile_avg_%s'%c1]))&(wb['推荐车型']!='C'),'yy_mile_avg_%s_g'%c1] = 5000
#     wb.loc[(~(pd.isna(wb['yy_mile_avg_%s'%c1])))&(wb['推荐车型']!='C'),'yy_mile_avg_%s_g'%c1] = 10000
    
    c1='机滤'
    wb['yy_day_avg_%s_g'%c1] = 365
    wb['yy_mile_avg_%s_g'%c1] = np.nan
    wb.loc[(pd.isna(wb['yy_mile_avg_%s'%c1]))&(wb['推荐车型']!='F'),'yy_mile_avg_%s_g'%c1] = 5000
    wb.loc[(~(pd.isna(wb['yy_mile_avg_%s'%c1])))&(wb['推荐车型']!='F'),'yy_mile_avg_%s_g'%c1] = 10000
    
#     c1='空滤'
#     wb['yy_day_avg_%s_g'%c1] = 365*2
#     wb['yy_mile_avg_%s_g'%c1] = 20000

    c1='燃油滤清器'
    wb.loc[wb['推荐车型'].isin(['B','J','E']),'yy_day_avg_%s_g'%c1] = np.nan
    wb.loc[wb['推荐车型'].isin(['B','J','E']),'yy_mile_avg_%s_g'%c1] = 60000
    
#     涡轮增压TSI
#     自然吸气MPI
#     c1='火花塞'
#     wb.loc[wb['推荐车型'].isin(['E']),'yy_day_avg_%s_g'%c1] = 365*2
#     wb.loc[wb['推荐车型'].isin(['A','L']),'yy_mile_avg_%s_g'%c1] = 30000
#     wb.loc[wb['推荐车型'].isin(['B','E','G','C','H','F','I','K']),'yy_mile_avg_%s_g'%c1] = 20000
#     wb.loc[(wb['推荐车型'].isin(['D','H','J']))&(wb['进气方式']=='涡轮增压'),'yy_mile_avg_%s_g'%c1] = 20000
#     wb.loc[(wb['推荐车型'].isin(['D','H','J']))&(wb['进气方式']=='自然吸气'),'yy_mile_avg_%s_g'%c1] = 30000    
    
    c1='变速箱油'
    wb.loc[(~wb['推荐车型'].isin(['F','G'])),'yy_day_avg_%s_g'%c1] = np.nan
    wb.loc[(~wb['推荐车型'].isin(['F','G'])),'yy_mile_avg_%s_g'%c1] = 60000
    
    c1='楔形皮带'
    wb['yy_day_avg_%s_g'%c1] =np.nan
    wb.loc[(wb['推荐车型'].isin(['A','B','E','G','H','J','K','M','N','O'])),'yy_mile_avg_%s_g'%c1] =100000
    wb.loc[(wb['推荐车型']=='D')&(wb['排量']=='2.0'),'yy_mile_avg_%s_g'%c1] = 100000 
    wb.loc[(wb['推荐车型']=='D')&(wb['排量']=='3.0'),'yy_mile_avg_%s_g'%c1] = 60000 
    
    c1='制动液'
    wb.loc[(pd.isna(wb['yy_mile_avg_%s'%c1]))&(wb['推荐车型'].isin(['A','B','C','D','G','H','K','M','N'])),'yy_day_avg_%s_g'%c1] =3
    wb.loc[(~(pd.isna(wb['yy_mile_avg_%s'%c1])))&(wb['推荐车型'].isin(['A','B','C','D','G','H','K','M','N'])),'yy_day_avg_%s_g'%c1] = 2
    wb.loc[(pd.isna(wb['yy_mile_avg_%s'%c1]))&(wb['推荐车型'].isin(['E','F','J','O']))&
           (wb['车辆类型']!='营运车'),'yy_day_avg_%s_g'%c1] =3
    wb.loc[(~(pd.isna(wb['yy_mile_avg_%s'%c1])))&(wb['推荐车型'].isin(['E','F','J','O']))
           &(wb['车辆类型']!='营运车'),'yy_day_avg_%s_g'%c1] = 2
    wb.loc[(wb['推荐车型'].isin(['E','F','J','O']))&
           (wb['车辆类型']=='营运车'),'yy_day_avg_%s_g'%c1] =2
    wb.loc[(wb['推荐车型'].isin(['E','F','J','O']))&
           (wb['车辆类型']=='营运车'),'yy_mile_avg_%s_g'%c1] =50000

#     c1='轮胎'
#     wb.loc[wb['推荐车型']=='C','yy_day_avg_%s_g'%c1] = 365/2
#     wb.loc[wb['推荐车型']!='C','yy_day_avg_%s_g'%c1] = 365
#     wb.loc[wb['推荐车型']=='C','yy_mile_avg_%s_g'%c1] = 5000
#     wb.loc[(pd.isna(wb['yy_mile_avg_%s'%c1])),'yy_mile_avg_%s_g'%c1] = 5000
#     wb.loc[(~(pd.isna(wb['yy_mile_avg_%s'%c1])))&(wb['推荐车型']!='C'),'yy_mile_avg_%s_g'%c1] = 10000

#     c1='电瓶/蓄电池'
#     wb.loc[wb['推荐车型']=='C','yy_day_avg_%s_g'%c1] = 365/2
#     wb.loc[wb['推荐车型']!='C','yy_day_avg_%s_g'%c1] = 365
#     wb.loc[wb['推荐车型']=='C','yy_mile_avg_%s_g'%c1] = 5000
#     wb.loc[(pd.isna(wb['yy_mile_avg_%s'%c1])),'yy_mile_avg_%s_g'%c1] = 5000
#     wb.loc[(~(pd.isna(wb['yy_mile_avg_%s'%c1])))&(wb['推荐车型']!='C'),'yy_mile_avg_%s_g'%c1] = 10000
    
    c1='正时皮带'
    wb.loc[(wb['车型档次']==1),'yy_mile_avg_%s_g'%c1]=64705
    wb.loc[(wb['车型档次']==1),'yy_day_avg_%s_g'%c1]=1584
    wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=82742.5
    wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=1763.5
    wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=128019
    wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=1660
    
#     c1='雨刮片'
#     wb['yy_mile_avg_%s_g'%c1] = 10000
#     wb['yy_day_avg_%s_g'%c1] = 365
    
#     c1='刹车片'
#     wb['yy_mile_avg_%s_g'%c1] = 10000
#     wb['yy_day_avg_%s_g'%c1] = 365
    
#     c1='刹车盘/制动盘'
#     wb['yy_mile_avg_%s_g'%c1] = 10000
#     wb['yy_day_avg_%s_g'%c1] = 365
    
#     c1='冷却液/防冻液' 
#     wb.loc[wb['推荐车型']=='C','yy_day_avg_%s_g'%c1] = 365/2
#     wb.loc[wb['推荐车型']!='C','yy_day_avg_%s_g'%c1] = 365
#     wb.loc[wb['推荐车型']=='C','yy_mile_avg_%s_g'%c1] = 5000
#     wb.loc[(pd.isna(wb['yy_mile_avg_%s'%c1])),'yy_mile_avg_%s_g'%c1] = 5000
#     wb.loc[(~(pd.isna(wb['yy_mile_avg_%s'%c1])))&(wb['推荐车型']!='C'),'yy_mile_avg_%s_g'%c1] = 10000
    
    c1='点火线圈'
    wb.loc[(wb['车型档次']==1),'yy_mile_avg_%s_g'%c1]=66115
    wb.loc[(wb['车型档次']==1),'yy_day_avg_%s_g'%c1]=1255
    wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=70478.5
    wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=1348
    wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=95549.5
    wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=1516.5
    wb.loc[(wb['车型档次']==4),'yy_mile_avg_%s_g'%c1]=41845
    wb.loc[(wb['车型档次']==4),'yy_day_avg_%s_g'%c1]=455
    
    c1='冷却系统养护'
    wb['yy_day_avg_%s_g'%c1] = 365*2
    wb['yy_mile_avg_%s_g'%c1] = 20000
#     wb.loc[(wb['车型档次']==1),'yy_mile_avg_%s_g'%c1]=44518
#     wb.loc[(wb['车型档次']==1),'yy_day_avg_%s_g'%c1]=1417
#     wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=50037.5
#     wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=1385.5
#     wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=55205.5
#     wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=1439
#     wb.loc[(wb['车型档次']==4),'yy_mile_avg_%s_g'%c1]=52531
#     wb.loc[(wb['车型档次']==4),'yy_day_avg_%s_g'%c1]=614
    
    c1='绿色养护'
    wb.loc[(wb['车型档次']==1),'yy_mile_avg_%s_g'%c1]=26456
    wb.loc[(wb['车型档次']==1),'yy_day_avg_%s_g'%c1]=920
    wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=30000
    wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=868
    wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=31449
    wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=867
    wb.loc[(wb['车型档次']==4),'yy_mile_avg_%s_g'%c1]=24983.5
    wb.loc[(wb['车型档次']==4),'yy_day_avg_%s_g'%c1]=400

    c1='减振器'
    wb.loc[(wb['车型档次']==1),'yy_mile_avg_%s_g'%c1]=28338
    wb.loc[(wb['车型档次']==1),'yy_day_avg_%s_g'%c1]=791
    wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=31586
    wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=679
    wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=39199.5
    wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=790
    wb.loc[(wb['车型档次']==4),'yy_mile_avg_%s_g'%c1]=18743
    wb.loc[(wb['车型档次']==4),'yy_day_avg_%s_g'%c1]=371
    
    c1='变速箱滤清器'
    wb.loc[(wb['车型档次']==1),'yy_mile_avg_%s_g'%c1]=88351
    wb.loc[(wb['车型档次']==1),'yy_day_avg_%s_g'%c1]=1253
    wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=66680.5
    wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=1155.5
    wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=65167.5
    wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=934.5
    wb.loc[(wb['车型档次']==4),'yy_mile_avg_%s_g'%c1]=62547
    wb.loc[(wb['车型档次']==4),'yy_day_avg_%s_g'%c1]=677
    
    c1='空调冷媒'
    wb.loc[(wb['车型档次']==1),'yy_mile_avg_%s_g'%c1]=30334
    wb.loc[(wb['车型档次']==1),'yy_day_avg_%s_g'%c1]=791
    wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=31778.5
    wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=757
    wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=42089
    wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=802
    wb.loc[(wb['车型档次']==4),'yy_mile_avg_%s_g'%c1]=20222
    wb.loc[(wb['车型档次']==4),'yy_day_avg_%s_g'%c1]=356.5

#     c1='转向助力液'
#     wb.loc[(wb['车型档次']==2),'yy_mile_avg_%s_g'%c1]=73520.5
#     wb.loc[(wb['车型档次']==2),'yy_day_avg_%s_g'%c1]=2391
#     wb.loc[(wb['车型档次']==3),'yy_mile_avg_%s_g'%c1]=84015
#     wb.loc[(wb['车型档次']==3),'yy_day_avg_%s_g'%c1]=2401
    
    rate = 0.8
    rate1=0.2
    for x in yisun_list:
        print(x)
        wb['yy_last_time_'+x] = pd.to_datetime(wb['yy_last_time_'+x])
        wb.loc[pd.isna(wb['yy_last_time_'+x]),'mile_gap_'+x] = wb['mile_next']
        wb.loc[~(pd.isna(wb['yy_last_time_'+x])),'mile_gap_'+x] = wb['mile_next'] - wb['yy_last_mile_'+x]
    
        wb['day_gap_0_'+x] = list(map(lambda x,y:(x-y).days,wb['next_repare_time_'],wb['yy_last_time_'+x]))
        wb['day_gap_1_'+x] = list(map(lambda x,y:(x-y).days,wb['next_repare_time_'],wb['purchase_date']))
        
        wb.loc[~(pd.isnull(wb['yy_last_time_'+x])),'day_gap_'+x] = wb['day_gap_0_'+x]
        wb.loc[pd.isnull(wb['yy_last_time_'+x]),'day_gap_'+x] = wb['day_gap_1_'+x]
#        对于有更换时间，却没有更换里程的vin，只用时间做判断,个人习惯小于官方建议*0.67，视为异常值，小于0，视为异常
        wb.loc[(~pd.isnull(wb['yy_last_time_'+x])) & (pd.isnull(wb['yy_last_time_'+x])),'mile_gap_'+x] = np.nan
       
        wb.loc[(wb['yy_mile_avg_'+x] < 0.67*wb['yy_mile_avg_'+x+'_g']) | (wb['yy_mile_avg_'+x] >1.5 *wb['yy_mile_avg_'+x+'_g']),'yy_mile_avg_'+x] = np.nan
        wb.loc[(wb['yy_day_avg_'+x] < 0.67*wb['yy_day_avg_'+x+'_g']) | (wb['yy_day_avg_'+x] >1.5*wb['yy_day_avg_'+x+'_g']),'yy_day_avg_'+x] = np.nan
       
        wb.loc[pd.isna(wb['yy_mile_avg_'+x+'_g']),'yy_mile_avg_'+x] = np.nan
        wb.loc[pd.isna(wb['yy_day_avg_'+x+'_g']) ,'yy_day_avg_'+x] = np.nan

        wb[x+'_mile_0'] = list(map(lambda mile_gap,avg_mile:(mile_gap%avg_mile>avg_mile*rate) or (mile_gap> avg_mile and mile_gap%avg_mile < avg_mile*rate1) ,wb['mile_gap_'+x],wb['yy_mile_avg_'+x]))
        wb[x+'_day_0'] = list(map(lambda day_gap,avg_day:(day_gap%avg_day>avg_day*rate) or (day_gap>avg_day and day_gap%avg_day<avg_day*rate1),wb['day_gap_'+x],wb['yy_day_avg_'+x]))
        wb[x+'_mile_1'] = list(map(lambda mile_gap,avg_mile:(mile_gap%avg_mile>avg_mile*rate) or (mile_gap> avg_mile and mile_gap%avg_mile < avg_mile*rate1) ,wb['mile_gap_'+x],wb['yy_mile_avg_'+x+'_g']))
        wb[x+'_day_1'] = list(map(lambda day_gap,avg_day:(day_gap%avg_day>avg_day*rate) or (day_gap>avg_day and day_gap%avg_day<avg_day*rate1),wb['day_gap_'+x],wb['yy_day_avg_'+x+'_g']))
        
        wb[x+'reason_mile_0'] = list(map(lambda x,y:'用户习惯,满'+str(round(y/10000,1))+'万公里' if x else '-',wb[x+'_mile_0'],wb['yy_mile_avg_'+x]))
        wb[x+'reason_mile_1'] = list(map(lambda x,y:'官方手册,满'+str(round(y/10000,1))+'万公里' if x else '-',wb[x+'_mile_1'],wb['yy_mile_avg_'+x+'_g']))
        wb[x+'reason_day_0'] = list(map(lambda x,y:'用户习惯,满'+str(int(y/365))+'年'if y>= 365 else '用户习惯,满'+str(int(y))+'天' if x else '-',wb[x+'_day_0'],wb['yy_day_avg_'+x]))
#        wb[x+'reason_day_0'] = list(map(lambda x,y:'用户习惯,满'+str(int(y/365))+'年' if x else '-',wb[x+'_day_0'],wb['yy_day_avg_'+x]))
        wb[x+'reason_day_1'] = list(map(lambda x,y:'官方手册,满'+str(int(y/365))+'年' if x else '-',wb[x+'_day_1'],wb['yy_day_avg_'+x+'_g']))
        
        wb[x+'_reason'] = wb[x+'reason_mile_1']
        wb.loc[wb[x+'_reason'] == '-',x+'_reason'] = wb[x+'reason_day_1']
        wb.loc[wb[x+'_reason'] == '-',x+'_reason'] = wb[x+'reason_mile_0']
        wb.loc[wb[x+'_reason'] == '-',x+'_reason'] = wb[x+'reason_day_0']
        wb[x] = list(map(lambda x,y,w,z:'是' if x|y|w|z else '否',wb[x+'_mile_0'],wb[x+'_day_0'],wb[x+'_mile_1'],wb[x+'_day_1']))
    
    col_list1=[]
    for col in yisun_list:
        if col!='正时皮带':
            wb.rename(columns={col:'%s_%s_是否推荐'%(col,ziduan_name)},inplace=True)
            col_list1.append('%s_%s_是否推荐'%(col,ziduan_name))
        else:
            wb.rename(columns={col:'%s/链条_%s_是否推荐'%(col,ziduan_name)},inplace=True)
            col_list1.append('%s/链条_%s_是否推荐'%(col,ziduan_name))
        
    col_list2=[]
    for col in yisun_list:
        if col!='正时皮带':
            wb.rename(columns={'%s_reason'%col:'%s_推荐理由_%s'%(col,ziduan_name)},inplace=True)
            col_list2.append('%s_推荐理由_%s'%(col,ziduan_name))
        else:
            wb.rename(columns={'%s_reason'%col:'%s/链条_推荐理由_%s'%(col,ziduan_name)},inplace=True)
            col_list2.append('%s/链条_推荐理由_%s'%(col,ziduan_name))            
    wb_s = wb[['VIN']+col_list1+col_list2]  
    gc.collect()
    return wb_s,wb

def get_concat(a,c,d,e,f,g,h,i,j,k,l,m):
    lst=[]
    if a=='是':
        lst.append('机滤')
    if c=='是':
        lst.append('燃油滤清器')
    if d=='是':
        lst.append('变速箱油')
    if e=='是':
        lst.append('楔形皮带')
    if f=='是':
        lst.append('制动液')
    if g=='是':
        lst.append('正时皮带/链条')
    if h=='是':
        lst.append('点火线圈')
    if i=='是':
        lst.append('冷却系统养护')
    if j=='是':
        lst.append('绿色养护')
    if k=='是':
        lst.append('减振器')
    if l=='是':
        lst.append('变速箱滤清器')
    if m=='是':
        lst.append('空调冷媒')
    return ','.join(lst)

# 润滑系统,5000公里
# 燃油系统,15000公里
# 进气系统,10000公里
# 冷却系统,每2年或2万公里保养使用一次,放到wb中计算
# 空调系统,春/夏/秋每季保养一次,其中冷气强化剂,春/夏交替时保养一次,这个只要气节符合,门店销售可以直接推荐
# 绿色养护,建议每月保养一次,门店销售可以直接推荐

def yisun_reminder(yisun_df,veh,veh3,pred,savep,updated_savep,today,yisun_list):
    
    veh1=veh[['VIN','技术代码','排量','进气方式','车辆类型']].drop_duplicates()
    print(veh1.shape)
    print(veh1['VIN'].nunique())

    veh2=veh[['VIN','一级车系','二级车系','车型档次']].drop_duplicates().reset_index(drop=True)
    print(veh2.shape)
    print(veh2['VIN'].nunique())
    
    veh3=veh3[['VIN','购车时间']].drop_duplicates().reset_index(drop=True)
    print(veh3.shape)
    print(veh3['VIN'].nunique())
    
    yisun_df=yisun_df.merge(veh1,on=['VIN'],how='left').merge(veh2,on=['VIN'],how='left').merge(veh3,on=['VIN'],how='left')
    yisun_df['购车时间']=pd.to_datetime(yisun_df['购车时间'])
    yisun_df=get_car_type(yisun_df)
    
    wbb=yanghu_yisun_interval(yisun_df,pd.to_datetime(today))

#     yisun_list=['机滤','燃油滤清器','变速箱油','楔形皮带','制动液','正时皮带','点火线圈','冷却系统养护',
#             '绿色养护','减振器','变速箱滤清器','空调冷媒']
    
    print('根据预测的公里数与天数差')
    s1=time.time()
    print(datetime.now())
    next_inshop1=pred[['VIN','下次保养时间_当前时间之后_按客户习惯', '下次保养里程_当前时间之后_按客户习惯']].drop_duplicates().reset_index(drop=True)
    next_inshop1.rename(columns={'下次保养里程_当前时间之后_按客户习惯':'mile_next',
                                 '下次保养时间_当前时间之后_按客户习惯':'next_repare_time_'},inplace=True)

    ziduan_name='模型预测'
    wb_s1,wb1= yisun_recommend(wbb,next_inshop1,yisun_list,yisun_df,ziduan_name)
    wb_s1=wb_s1.drop_duplicates().reset_index(drop=True)
    wb_s1.to_pickle(savep+'易损件提醒_%s.pkl'%(yisun_list[0]))
    print('run time is:',(time.time()-s1)/60)
    
    # wb1=wb1.drop_duplicates().reset_index(drop=True)
    # wb1.to_excel(savep+'易损件提醒check_%s.xlsx'%(dt),index = False,encoding='gbk')
    
    print('='*40)
    print('根据预测的日均与养护官方建议的')
    s1=time.time()
    print(datetime.now())
    next_inshop2=pred[['VIN','下次保养时间_当前时间之后_按官方建议周期','下次保养里程_当前时间之后_按官方建议周期']].drop_duplicates().reset_index(drop=True)
    next_inshop2.rename(columns={'下次保养里程_当前时间之后_按官方建议周期':'mile_next',
                                 '下次保养时间_当前时间之后_按官方建议周期':'next_repare_time_'},inplace=True)

    ziduan_name='根据每1万公里保养建议'
    wb_s2,wb2=yisun_recommend(wbb,next_inshop2,yisun_list,yisun_df,ziduan_name)
    wb_s2=wb_s2.drop_duplicates().reset_index(drop=True)
    wb_s2.to_pickle(savep+'易损件提醒_1万养护周期_%s.pkl'%(yisun_list[0]))  
    print('run time is:',(time.time()-s1)/60)
    # wb2=wb2.drop_duplicates().reset_index(drop=True)
    # wb2.to_excel(savep+'易损件提醒check_1万养护周期_%s.xlsx'%(dt),index = False,encoding='gbk')

    return wb_s1,wb_s2

In [34]:
yisun_df=pd.read_pickle(savep+'yanghu_yisun_df.pkl')
veh=pd.read_pickle(savep+'vehicle_info.pkl')
veh3=pd.read_pickle(savep+'final_zhudan.pkl')
pred=pd.read_pickle(savep+'2个预测结果_用于易损件推荐.pkl')

In [35]:
yisun_list=['机滤','燃油滤清器','变速箱油','楔形皮带','制动液','正时皮带','点火线圈','冷却系统养护',
            '绿色养护','减振器','变速箱滤清器','空调冷媒']

In [36]:
wb_s1,wb_s2=yisun_reminder(yisun_df,veh,veh3,pred,savep,updated_savep,today,yisun_list)

(850310, 5)
850310
(850310, 4)
850310
(804143, 2)
804143
根据预测的公里数与天数差
2020-10-24 12:03:26.879560
wbb表格的情况  (773925, 141)
wb表格的情况  (773925, 143)
机滤
燃油滤清器
变速箱油
楔形皮带
制动液
正时皮带
点火线圈
冷却系统养护
绿色养护
减振器
变速箱滤清器
空调冷媒
run time is: 230.11751805146534
根据预测的日均与养护官方建议的
2020-10-24 15:53:33.931598
wbb表格的情况  (773925, 141)
wb表格的情况  (773925, 143)
机滤
燃油滤清器
变速箱油
楔形皮带
制动液
正时皮带
点火线圈
冷却系统养护
绿色养护
减振器
变速箱滤清器
空调冷媒
run time is: 220.20942145586014


In [37]:
wb_s1['下次产品更换提醒_当前时间之后_按客户习惯']=list(map(lambda a,c,d,e,f,g,h,i,j,k,l,m:get_concat(a,c,d,e,f,\
                                                                                   g,h,i,j,k,l,m),\
                                         wb_s1['机滤_模型预测_是否推荐'],wb_s1['燃油滤清器_模型预测_是否推荐'],\
                                         wb_s1['变速箱油_模型预测_是否推荐'],wb_s1['楔形皮带_模型预测_是否推荐'],\
                                         wb_s1['制动液_模型预测_是否推荐'],wb_s1['正时皮带/链条_模型预测_是否推荐'],\
                                         wb_s1['点火线圈_模型预测_是否推荐'],wb_s1['冷却系统养护_模型预测_是否推荐'],\
                                         wb_s1['绿色养护_模型预测_是否推荐'],wb_s1['减振器_模型预测_是否推荐'],\
                                         wb_s1['变速箱滤清器_模型预测_是否推荐'],wb_s1['空调冷媒_模型预测_是否推荐']))
wb_s1.loc[wb_s1['下次产品更换提醒_当前时间之后_按客户习惯']=='','下次产品更换提醒_当前时间之后_按客户习惯']='-'
wb_s1[['下次产品更换提醒_当前时间之后_按客户习惯']]

,下次产品更换提醒_当前时间之后_按客户习惯
0,"制动液,冷却系统养护"
1,"机滤,燃油滤清器,变速箱油"
2,"机滤,楔形皮带,冷却系统养护"
3,"机滤,冷却系统养护"
4,"机滤,变速箱油,楔形皮带"
...,...
773920,"机滤,楔形皮带"
773921,"机滤,冷却系统养护"
773922,机滤
773923,机滤


In [38]:
gc.collect()
    
wb_s2['下次产品更换提醒_当前时间之后_按官方建议周期']=list(map(lambda a,c,d,e,f,g,h,i,j,k,l,m:get_concat(a,c,d,\
                                                                                     e,f,g,h,i,j,k,l,m),\
                                           wb_s2['机滤_根据每1万公里保养建议_是否推荐'],\
                                           wb_s2['燃油滤清器_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['变速箱油_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['楔形皮带_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['制动液_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['正时皮带/链条_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['点火线圈_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['冷却系统养护_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['绿色养护_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['减振器_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['变速箱滤清器_根据每1万公里保养建议_是否推荐'],
                                           wb_s2['空调冷媒_根据每1万公里保养建议_是否推荐']))
wb_s2.loc[wb_s2['下次产品更换提醒_当前时间之后_按官方建议周期']=='','下次产品更换提醒_当前时间之后_按官方建议周期']='-'
wb_s2[['下次产品更换提醒_当前时间之后_按官方建议周期']]

,下次产品更换提醒_当前时间之后_按官方建议周期
0,机滤
1,"机滤,冷却系统养护"
2,冷却系统养护
3,"机滤,变速箱油,制动液"
4,"楔形皮带,冷却系统养护"
...,...
773920,"燃油滤清器,变速箱油,制动液"
773921,机滤
773922,"机滤,制动液"
773923,机滤


In [39]:
gc.collect()
wbb=wb_s1[['VIN','下次产品更换提醒_当前时间之后_按客户习惯']].merge(wb_s2[['VIN','下次产品更换提醒_当前时间之后_按官方建议周期']],on=['VIN'],how='left')
print(wbb.shape)
print(wbb['VIN'].nunique())

(773925, 3)
773925


In [40]:
wbb.to_pickle(savep+'下次产品更换提醒.pkl')
wbb.to_pickle(updated_savep+'下次产品更换提醒.pkl')

In [41]:
print('run time is: ',(time.time()-s1)/60)

run time is:  482.1859161496162


In [42]:
482/60

8.033333333333333